### Import Libraries

In [1]:
import pandas as pd # Dataset operations
import numpy as np # math and array operations
from scipy.spatial import distance

### Import Dataset
Import dataset with the following columns:
- Propierties
- Units
- Max or Min Criteria
- Next columns are for each material with its properties values

In [2]:
df = pd.read_excel("Materiales.xlsx")
df.head()

,Propiedad,Unidades,Criterio,PP,LDPE,HDPE,N6
0,Esfuerzo fluencia,Mpa,Max,41.40,78.000,38.00,79.00
1,Módulo de elasticidad,Gpa,Max,1.77,0.380,1.50,2.90
2,Densidad,g/cm^3,Min,0.92,0.925,0.96,1.14
3,Absorción de agua,%,Min,0.02,0.015,0.20,1.80
4,Costo de la materia prima,USD/kg,Min,0.97,1.060,0.90,2.10


In [3]:
# List of criterias
criterias = df["Propiedad"].tolist()
criterias

['Esfuerzo fluencia',
 'Módulo de elasticidad',
 'Densidad',
 'Absorción de agua',
 'Costo de la materia prima']

In [4]:
# List of alternatives
alternatives =  df.columns.tolist()[3:]
alternatives

['PP', 'LDPE', 'HDPE', 'N6']

In [5]:
# Max or min criterias
maxmin = df["Criterio"].tolist()
maxmin

['Max', 'Max', 'Min', 'Min', 'Min']

# TOPSIS

### Decisional matrix

In [6]:
df.pop("Unidades") # Deleta Units column
df.pop("Criterio") # Delete max-min criteria colum
df.pop("Propiedad")# Delete properties column
df

,PP,LDPE,HDPE,N6
0,41.40,78.000,38.00,79.00
1,1.77,0.380,1.50,2.90
2,0.92,0.925,0.96,1.14
3,0.02,0.015,0.20,1.80
4,0.97,1.060,0.90,2.10


In [7]:
df_decisional = df.T
df_decisional.columns = criterias
df_decisional

,Esfuerzo fluencia,Módulo de elasticidad,Densidad,Absorción de agua,Costo de la materia prima
PP,41.4,1.77,0.920,0.020,0.97
LDPE,78.0,0.38,0.925,0.015,1.06
HDPE,38.0,1.50,0.960,0.200,0.90
N6,79.0,2.90,1.140,1.800,2.10


In [8]:
np.set_printoptions(suppress=True, precision=5) # Avoid scientific notation in numpy matrix and 3 decimal digits

m_decisional = df_decisional.to_numpy() #Dataframe into numpy matrix
m_decisional

array([[41.4  ,  1.77 ,  0.92 ,  0.02 ,  0.97 ],
       [78.   ,  0.38 ,  0.925,  0.015,  1.06 ],
       [38.   ,  1.5  ,  0.96 ,  0.2  ,  0.9  ],
       [79.   ,  2.9  ,  1.14 ,  1.8  ,  2.1  ]])

### r_ij^2  Matrix

#### Normalized
$$x_{ij} = {r_{ij}\over {\sqrt{\sum{r_{ij}^2}}}}$$

In [9]:
m_rij = m_decisional**2 # Matrix to save r_ij^2  values

sum_column = np.zeros(len(criterias)) # Array to save columns sum

for i in range(len(criterias)):
    
    sum_column[i] = np.sum(m_rij[:,i])

print("r_ij squared Matrix")
print(m_rij)
print(" ")
print("Sum of each column")
print(sum_column)

r_ij squared Matrix
[[1713.96       3.1329     0.8464     0.0004     0.9409 ]
 [6084.         0.1444     0.85563    0.00022    1.1236 ]
 [1444.         2.25       0.9216     0.04       0.81   ]
 [6241.         8.41       1.2996     3.24       4.41   ]]
 
Sum of each column
[15482.96       13.9373      3.92322     3.28063     7.2845 ]


### Normalized Matrix

In [10]:
m_normalized = m_decisional/np.sqrt(sum_column) # Matrix to save normalized values
print(m_normalized)

[[0.33272 0.47412 0.46448 0.01104 0.35939]
 [0.62686 0.10179 0.467   0.00828 0.39274]
 [0.30539 0.40179 0.48467 0.11042 0.33346]
 [0.63489 0.7768  0.57555 0.99379 0.77807]]


### Weights

In [11]:
w = np.zeros(len(criterias)) # Array to save criterias' weight

condition = True # Cndition sum weights = 1
 
while condition:
    
    for i in range(len(criterias)):
        w[i] = float(input("Weight of {}: ".format(criterias[i])))
    
    if np.sum(w) == 1:
        condition = False
    else:
        print(" ")
        print("¡Sum of weights is different to 1, correct the values!")

print(" ")
list(zip(criterias,w))

Weight of Esfuerzo fluencia: 0.1
Weight of Módulo de elasticidad: 0.15
Weight of Densidad: 0.3
Weight of Absorción de agua: 0.1
Weight of Costo de la materia prima: 0.35
 


[('Esfuerzo fluencia', 0.1),
 ('Módulo de elasticidad', 0.15),
 ('Densidad', 0.3),
 ('Absorción de agua', 0.1),
 ('Costo de la materia prima', 0.35)]

### Weighted matrix

In [12]:
m_weight = m_normalized*w # Matrix to save weighted values
print(m_weight)

[[0.03327 0.07112 0.13934 0.0011  0.12579]
 [0.06269 0.01527 0.1401  0.00083 0.13746]
 [0.03054 0.06027 0.1454  0.01104 0.11671]
 [0.06349 0.11652 0.17267 0.09938 0.27232]]


### Ideal and Non-Ideal alternatives

In [13]:
v_max = np.zeros(len(criterias)) # Arrays to save the ideal alternative
v_min = np.zeros(len(criterias)) # Arrays to save the Non-ideal alternative

for i in range(len(criterias)):
    
    if maxmin[i] == "Max":
        v_max[i] = max(m_weight[:,i])
        v_min[i] = min(m_weight[:,i])
    
    elif maxmin[i] == "Min":
        v_max[i] = min(m_weight[:,i])
        v_min[i] = max(m_weight[:,i])
        
print("V+")
print(v_max)
print(" ")
print("V-")
print(v_min)

V+
[0.06349 0.11652 0.13934 0.00083 0.11671]
 
V-
[0.03054 0.01527 0.17267 0.09938 0.27232]


### Euclidean Distances

In [14]:
dist_max = np.zeros(len(alternatives)) # Euclidean distance to the ideal solution
dist_min = np.zeros(len(alternatives)) # Euclidean distance to the Non-ideal solution

for i in range(len(alternatives)):
    dist_max[i] = distance.euclidean(m_weight[i,:],v_max)
    dist_min[i] = distance.euclidean(m_weight[i,:],v_min)

print("d+")
print(dist_max)
print(" ")
print("d-")
print(dist_min)

d+
[0.05529 0.10336 0.06626 0.18719]
 
d-
[0.18806 0.17319 0.18651 0.10648]


### Relative Proximity

In [15]:
r_prox = dist_min/(dist_max+dist_min)
print(r_prox)

[0.7728  0.62625 0.73786 0.36259]


### Results

In [16]:
# Show it as a Dataframe
df_end = pd.DataFrame(r_prox)
df_end.insert(0, "Material", alternatives)
df_end.columns = ["Material","Relative Proximity"]
df_end

,Material,Relative Proximity
0,PP,0.772800
1,LDPE,0.626248
2,HDPE,0.737857
3,N6,0.362586


In [17]:
df_end = df_end.sort_values(by=["Relative Proximity"], ascending = False)
df_end.insert(0, "Ranking", list(range(1,len(alternatives)+1)))
df_end

,Ranking,Material,Relative Proximity
0,1,PP,0.772800
2,2,HDPE,0.737857
1,3,LDPE,0.626248
3,4,N6,0.362586
